In [141]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'osm_join_small_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/osm_join_small_1/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/osm_join_small_1/outputs/results/run1', '/home/chesetti/Repos/learned_merge_cleanup/sponge/osm_join_small_1/outputs/results/run2']


In [ ]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'result.outer_index_build_duration',
       'result.outer_index_size', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [ ]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name   btree_1page  btree_2page  btree_4page   pgm1024    pgm128  \
spec.common_key                                                              
10                  41676380     40690489     40077542  38437773  32109143   
20                  30153093     27965332     28761239  31811786  23225935   
30                  24662293     22495130     21760602  27949996  18927381   
40                  21946629     19075091     18850344  27267656  16821837   
50                  20408562     17230961     17135105  25744539  15882820   
60                  18203819     16357186     15391568  24927726  15331196   
70                  16737289     15120268     14050359  23440757  14427015   
80                  15943737     13992025     13026946  22801559  13480076   
90                  15371795     13365125     12247476  22615326  13211882   
100                 15140680     12773008     11803872  21422115  12702784   

spec.algo_name     pgm256    pgm512     pgm64        sj       s

In [154]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

       indexes  memory_bytes
0  btree_1page       70032.0
1  btree_2page       37072.0
2  btree_4page       20592.0
3      pgm1024        2320.0
4       pgm128       16328.0
5       pgm256        8480.0
6       pgm512        4352.0
7        pgm64       31400.0


In [ ]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [ ]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   btree_1page  btree_2page  btree_4page  pgm1024  pgm128  \
spec.common_key                                                           
10                      3903         1951          975     3282    3896   
20                      3903         1951          975     3200    3887   
30                      3902         1951          975     3135    3865   
40                      3899         1951          975     3073    3809   
50                      3880         1951          975     3017    3792   
60                      3850         1950          975     2937    3717   
70                      3797         1950          975     2854    3646   
80                      3729         1949          975     2795    3566   
90                      3647         1945          975     2722    3521   
100                     3576         1943          975     2648    3400   

spec.algo_name   pgm256  pgm512  pgm64    sj   sj2  
spec.common_key                               

In [143]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 30])
print(result_checksum.loc[result_checksum['spec.common_key'] == 30]['result.checksum'].unique())

common_key: 10 checksums don't match
common_key: 20 checksum: ['6B5439DA4F484C4C0DCFABD6AB6F2D2A'] OK
common_key: 30 checksums don't match
common_key: 40 checksum: ['E0078BDC49A86965006193905C392F63'] OK
common_key: 50 checksum: ['DE05035C7EA40B5B9447602CB5075768'] OK
common_key: 60 checksum: ['C8D4C8C21A57FB0020BA0A0EC81F85A7'] OK
common_key: 70 checksum: ['FA7542F3E03097C8168A2D268C01BA0D'] OK
common_key: 80 checksum: ['EDD76F678D96E487E2D8DA1F5F1BB5C8'] OK
common_key: 90 checksum: ['2E9A96243F4D65D4A1AC356ADA3B7BF8'] OK
common_key: 100 checksum: ['24A07FF9357C9ABBB64688C560D08177'] OK
     spec.common_key spec.algo_name                   result.checksum
47                30    btree_1page  B482C906A3A265DC0FD1728D68A78C41
147               30    btree_1page  B482C906A3A265DC0FD1728D68A78C41
247               30    btree_1page  B482C906A3A265DC0FD1728D68A78C41
18                30    btree_2page  B482C906A3A265DC0FD1728D68A78C41
118               30    btree_2page  B482C906A3A265DC0F